In [1]:
# boilerplate
import pandas as pd
import xlsxwriter

path_to_input = '../../output/'
path_to_output = '../../output/'

In [2]:
# load data
df1 = pd.read_csv(path_to_input + 'ctgov_search1and2_intersect_deduplicated.csv')
df2 = pd.read_csv(path_to_input + 'ictrp_search1and2_intersect_deduplicated.csv')

# concatenate and deduplicate
df_combo = pd.concat([df1, df2], axis=0)
df_combo.drop_duplicates(subset='TrialID', keep='first', inplace=True)

# cleanup
df_combo.sort_values(by='Register', inplace=True)
df_combo.reset_index(drop=True, inplace=True)

# save data
df_combo.to_csv(path_to_output + 'ct_list.csv', index=False)
df_combo.to_excel(path_to_output + 'ct_list.xlsx', index=False)

In [3]:
# Export to xlsx file
df_combo.to_excel(path_to_output + 'ct_list.xlsx', engine='xlsxwriter') 

#Open it with xlsxwriter
writer = pd.ExcelWriter(path_to_output + 'ct_list.xlsx', engine='xlsxwriter') 
df_combo.to_excel(writer, sheet_name='Sheet1')

#Assign the workbook and worksheet
workbook  = writer.book
worksheet = writer.sheets['Sheet1']

# add new column for screening
cell_format_colname = workbook.add_format({'bold': True})
worksheet.write(0, 5, 'Screening', cell_format_colname)


# reformat URL column so that the horizontal alignment is set to fill (stop text overflow)
cell_format_coldata = workbook.add_format()
cell_format_coldata.set_align('fill')

last_row = len(df_combo)

for row in range(0, last_row):

    trialid_ = df_combo.iloc[row, 0]
    title_ = df_combo.iloc[row, 1]
    register_ = df_combo.iloc[row, 2]
    url_ = df_combo.iloc[row, 3]

    row_reindex = row + 1

    worksheet.write(row_reindex, 1, trialid_, cell_format_coldata)
    worksheet.write(row_reindex, 2, title_, cell_format_coldata)
    worksheet.write(row_reindex, 3, register_, cell_format_coldata)
    worksheet.write(row_reindex, 4, url_, cell_format_coldata)

    worksheet.data_validation(row_reindex, 5, row_reindex, 5, {'validate': 'list',
                                  'source': ['Include',' Exclude']})


# close and save
workbook.close()